In [1]:
!pip install lazypredict


In [2]:
import pandas as pd

from sklearn.preprocessing import MinMaxScaler

In [3]:
fname_url="https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip"
fname=fname_url.split("/")[-1]

In [4]:
!wget $fname_url


--2021-12-07 21:08:12--  https://github.com/sipocz/Blood/raw/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip [following]
--2021-12-07 21:08:12--  https://raw.githubusercontent.com/sipocz/Blood/b5fc44c487b0a712d00865014203349fb8690257/orig/train.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16979362 (16M) [application/zip]
Saving to: ‘train.zip’

train.zip           100%[===================>]  16.19M  --.-KB/s    in 0.1s    

2021-12-07 21:08:13 (123 MB/s) - ‘trai

In [5]:
!mkdir "train"
!rm ./train/Train.csv

mkdir: cannot create directory ‘train’: File exists


In [6]:
!unzip $fname -d "./train"
!rm train.zip

Archive:  train.zip
  inflating: ./train/Train.csv       


In [60]:
fname_train="./train/Train.csv"
df=pd.read_csv(fname_train)

In [61]:
df.head(5)

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human
0,ID_3SSHI56C,0.48,0.48,0.49,0.49,0.50,0.50,0.51,0.51,0.52,0.54,0.55,0.56,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,...,1.47,1.46,1.45,1.44,1.42,1.40,1.39,1.38,1.36,1.36,1.34,1.34,1.32,1.32,1.31,1.29,1.29,1.29,1.28,1.27,1.26,1.25,1.25,1.26,1.25,1.24,1.24,1.24,1.23,1.23,1.22,1.21,1.20,1.20,1.18,42.51,34.01,ok,ok,ok
1,ID_599OOLZA,0.47,0.47,0.48,0.49,0.49,0.50,0.50,0.51,0.52,0.53,0.55,0.56,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.54,0.54,0.54,0.55,0.55,...,1.55,1.54,1.53,1.52,1.50,1.49,1.47,1.47,1.45,1.44,1.42,1.42,1.40,1.40,1.38,1.38,1.37,1.36,1.35,1.34,1.34,1.33,1.34,1.34,1.33,1.33,1.34,1.33,1.31,1.31,1.30,1.32,1.31,1.26,1.26,44.52,32.09,ok,high,high
2,ID_MVJGPQ75,0.44,0.46,0.45,0.46,0.46,0.48,0.48,0.48,0.49,0.51,0.52,0.53,0.54,0.54,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.53,0.53,0.53,0.53,...,1.52,1.50,1.49,1.48,1.46,1.45,1.45,1.44,1.42,1.42,1.40,1.39,1.40,1.39,1.38,1.36,1.35,1.35,1.33,1.33,1.34,1.33,1.34,1.32,1.33,1.33,1.32,1.31,1.30,1.29,1.33,1.27,1.34,1.29,1.35,45.77,24.80,ok,ok,high
3,ID_CK6RF8YV,0.51,0.51,0.52,0.52,0.52,0.53,0.54,0.55,0.55,0.57,0.58,0.59,0.60,0.60,0.61,0.61,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.56,0.57,0.57,0.57,0.57,...,1.44,1.42,1.41,1.40,1.39,1.38,1.37,1.35,1.34,1.33,1.33,1.32,1.31,1.29,1.29,1.28,1.28,1.27,1.27,1.26,1.25,1.25,1.25,1.24,1.25,1.25,1.25,1.27,1.26,1.29,1.30,1.30,1.31,1.30,1.35,45.84,36.93,low,ok,high
4,ID_82N6QE6I,0.51,0.52,0.52,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.59,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.60,0.60,0.60,0.60,0.60,0.61,0.61,...,1.43,1.43,1.42,1.41,1.41,1.40,1.39,1.38,1.37,1.36,1.35,1.35,1.35,1.33,1.33,1.32,1.32,1.32,1.30,1.30,1.29,1.30,1.29,1.29,1.29,1.29,1.29,1.30,1.27,1.27,1.27,1.29,1.34,1.32,1.35,38.92,23.88,ok,ok,high


In [62]:
def one_hot_encode_df(df,str_index):
    from sklearn.preprocessing import OneHotEncoder
    indf=df.copy(True)
    one_hot=OneHotEncoder()
    col=indf[str_index].values
    col=col.reshape(-1,1)
    one_hot.fit(col)
    out=one_hot.transform(col).toarray()
    codes=one_hot.get_feature_names()
    print(codes)
    for ind,code in enumerate(codes):
        print("Itt:",ind,codes[ind])
        indf[str_index+"_"+code]=out[:,ind]
    return(indf,one_hot)

In [63]:
df,hdl_onehot=one_hot_encode_df(df,"hdl_cholesterol_human")
df,hemo_onehot=one_hot_encode_df(df,"hemoglobin(hgb)_human")
df,col_onehot=one_hot_encode_df(df,"cholesterol_ldl_human")

['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok
['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok
['x0_high' 'x0_low' 'x0_ok']
Itt: 0 x0_high
Itt: 1 x0_low
Itt: 2 x0_ok


In [64]:
df.tail()

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,hdl_cholesterol_human_x0_high,hdl_cholesterol_human_x0_low,hdl_cholesterol_human_x0_ok,hemoglobin(hgb)_human_x0_high,hemoglobin(hgb)_human_x0_low,hemoglobin(hgb)_human_x0_ok,cholesterol_ldl_human_x0_high,cholesterol_ldl_human_x0_low,cholesterol_ldl_human_x0_ok
13135,ID_NGPC0DA3,0.48,0.48,0.49,0.50,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.57,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.56,...,1.30,1.29,1.30,1.28,1.28,1.27,1.26,1.26,1.25,1.25,1.23,1.23,1.24,1.23,1.22,1.22,1.22,1.22,1.21,1.21,1.20,1.19,1.18,1.21,1.19,1.19,34.79,36.93,high,ok,ok,1.00,0.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
13136,ID_XRBUD5U8,0.53,0.53,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.60,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.58,0.59,0.59,0.60,...,1.48,1.46,1.44,1.45,1.43,1.42,1.42,1.41,1.42,1.41,1.39,1.38,1.38,1.38,1.38,1.38,1.38,1.36,1.38,1.38,1.37,1.39,1.38,1.37,1.40,1.39,43.12,19.14,ok,ok,ok,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
13137,ID_2M9L5NV2,0.51,0.52,0.52,0.53,0.54,0.54,0.55,0.56,0.57,0.58,0.60,0.61,0.62,0.62,0.63,0.63,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.60,0.60,...,1.45,1.43,1.43,1.42,1.41,1.42,1.40,1.40,1.39,1.38,1.38,1.36,1.36,1.37,1.37,1.37,1.36,1.37,1.36,1.39,1.39,1.40,1.38,1.43,1.45,1.38,42.48,43.41,ok,ok,ok,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
13138,ID_C5V5SD2D,0.46,0.47,0.47,0.47,0.48,0.49,0.49,0.50,0.51,0.52,0.53,0.54,0.55,0.56,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.54,0.54,0.54,...,1.29,1.29,1.28,1.27,1.26,1.26,1.25,1.25,1.24,1.23,1.22,1.22,1.22,1.23,1.22,1.21,1.21,1.21,1.22,1.21,1.19,1.18,1.17,1.17,1.15,1.21,41.86,35.10,ok,ok,ok,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
13139,ID_9R5ZR9NX,0.50,0.50,0.51,0.51,0.52,0.52,0.53,0.54,0.55,0.56,0.57,0.59,0.59,0.59,0.60,0.60,0.60,0.60,0.59,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.56,0.57,0.57,0.57,0.57,0.58,0.58,...,1.47,1.46,1.45,1.43,1.43,1.42,1.41,1.41,1.40,1.39,1.38,1.39,1.38,1.37,1.36,1.37,1.37,1.37,1.37,1.38,1.39,1.38,1.40,1.44,1.35,1.42,42.91,26.30,high,ok,high,1.00,0.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00


In [79]:
df_list=pd.DataFrame()
df_list["data"]=df["hdl_cholesterol_human"]+"+"+  df["hemoglobin(hgb)_human"]+"+"+df["cholesterol_ldl_human"]

AttributeError: ignored

In [94]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder

_3_Label_encoder=LabelEncoder()
_3_Label_encoder.fit(df_list)
_3_Label_encoder.classes_
y=_3_Label_encoder.transform(df_list)

In [96]:

del df["hdl_cholesterol_human"]
del df["hemoglobin(hgb)_human"]
del df["cholesterol_ldl_human"]




KeyError: ignored

In [97]:
from lazypredict.Supervised import LazyRegressor
from lazypredict.Supervised import LazyClassifier

from sklearn import datasets
from sklearn.utils import shuffle
import numpy as np


In [98]:
col_names=df.columns[-9:]

In [99]:
col_names[0]

'hdl_cholesterol_human_x0_high'

In [100]:
X=df.iloc[:,1:-9]
y

array([18, 12, 16, ..., 18, 18,  2])

In [101]:
y

array([18, 12, 16, ..., 18, 18,  2])

In [102]:
X

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity
0,0.48,0.48,0.49,0.49,0.50,0.50,0.51,0.51,0.52,0.54,0.55,0.56,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.55,...,1.52,1.51,1.49,1.47,1.46,1.45,1.44,1.42,1.40,1.39,1.38,1.36,1.36,1.34,1.34,1.32,1.32,1.31,1.29,1.29,1.29,1.28,1.27,1.26,1.25,1.25,1.26,1.25,1.24,1.24,1.24,1.23,1.23,1.22,1.21,1.20,1.20,1.18,42.51,34.01
1,0.47,0.47,0.48,0.49,0.49,0.50,0.50,0.51,0.52,0.53,0.55,0.56,0.57,0.57,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.54,0.54,0.54,0.55,0.55,0.55,...,1.62,1.59,1.58,1.55,1.54,1.53,1.52,1.50,1.49,1.47,1.47,1.45,1.44,1.42,1.42,1.40,1.40,1.38,1.38,1.37,1.36,1.35,1.34,1.34,1.33,1.34,1.34,1.33,1.33,1.34,1.33,1.31,1.31,1.30,1.32,1.31,1.26,1.26,44.52,32.09
2,0.44,0.46,0.45,0.46,0.46,0.48,0.48,0.48,0.49,0.51,0.52,0.53,0.54,0.54,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.53,0.53,0.53,0.53,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.52,0.53,0.53,0.53,0.53,0.54,...,1.54,1.54,1.53,1.52,1.50,1.49,1.48,1.46,1.45,1.45,1.44,1.42,1.42,1.40,1.39,1.40,1.39,1.38,1.36,1.35,1.35,1.33,1.33,1.34,1.33,1.34,1.32,1.33,1.33,1.32,1.31,1.30,1.29,1.33,1.27,1.34,1.29,1.35,45.77,24.80
3,0.51,0.51,0.52,0.52,0.52,0.53,0.54,0.55,0.55,0.57,0.58,0.59,0.60,0.60,0.61,0.61,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.56,0.57,0.57,0.57,0.57,0.58,...,1.48,1.47,1.46,1.44,1.42,1.41,1.40,1.39,1.38,1.37,1.35,1.34,1.33,1.33,1.32,1.31,1.29,1.29,1.28,1.28,1.27,1.27,1.26,1.25,1.25,1.25,1.24,1.25,1.25,1.25,1.27,1.26,1.29,1.30,1.30,1.31,1.30,1.35,45.84,36.93
4,0.51,0.52,0.52,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.59,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.60,0.60,0.60,0.60,0.60,0.61,0.61,0.61,...,1.46,1.46,1.45,1.43,1.43,1.42,1.41,1.41,1.40,1.39,1.38,1.37,1.36,1.35,1.35,1.35,1.33,1.33,1.32,1.32,1.32,1.30,1.30,1.29,1.30,1.29,1.29,1.29,1.29,1.29,1.30,1.27,1.27,1.27,1.29,1.34,1.32,1.35,38.92,23.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13135,0.48,0.48,0.49,0.50,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.57,0.57,0.58,0.57,0.57,0.57,0.57,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.56,0.56,...,1.44,1.43,1.41,1.41,1.39,1.38,1.37,1.36,1.34,1.33,1.33,1.31,1.30,1.29,1.30,1.28,1.28,1.27,1.26,1.26,1.25,1.25,1.23,1.23,1.24,1.23,1.22,1.22,1.22,1.22,1.21,1.21,1.20,1.19,1.18,1.21,1.19,1.19,34.79,36.93
13136,0.53,0.53,0.53,0.53,0.54,0.55,0.55,0.56,0.57,0.58,0.60,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0

In [103]:
offset=int(0.9*len(X))
print(offset)

11826


In [104]:
X_train, y_train = X[:offset], y[:offset]
X_test, y_test = X[offset:], y[offset:]

clf = LazyClassifier(verbose=1,ignore_warnings=True, custom_metric=None,predictions=True)
modells,predictions = clf.fit(X_train, X_test, y_train, y_test)


  3%|▎         | 1/29 [00:19<08:54, 19.08s/it]

{'Model': 'AdaBoostClassifier', 'Accuracy': 0.3082191780821918, 'Balanced Accuracy': 0.0881161713107252, 'ROC AUC': None, 'F1 Score': 0.20457526872083198, 'Time taken': 19.08021378517151}


  7%|▋         | 2/29 [00:41<09:33, 21.25s/it]

{'Model': 'BaggingClassifier', 'Accuracy': 0.9885844748858448, 'Balanced Accuracy': 0.9675056402573597, 'ROC AUC': None, 'F1 Score': 0.9885110699693824, 'Time taken': 22.758439540863037}


 10%|█         | 3/29 [00:42<05:02, 11.65s/it]

{'Model': 'BernoulliNB', 'Accuracy': 0.1050228310502283, 'Balanced Accuracy': 0.4000476219206964, 'ROC AUC': None, 'F1 Score': 0.07975450452560849, 'Time taken': 0.22481536865234375}


 10%|█         | 3/29 [02:32<22:02, 50.85s/it]


KeyboardInterrupt: ignored

In [105]:
modells

,Accuracy,Balanced Accuracy,ROC AUC,F1 Score,Time Taken
Model,,,,,
LabelSpreading,1.00,1.00,1.00,1.00,10.72
KNeighborsClassifier,1.00,1.00,1.00,1.00,0.96
LabelPropagation,1.00,1.00,1.00,1.00,6.49
ExtraTreesClassifier,1.00,1.00,1.00,1.00,2.25
LGBMClassifier,1.00,0.99,0.99,1.00,4.69
XGBClassifier,0.99,0.98,0.98,0.99,19.90
RandomForestClassifier,0.99,0.98,0.98,0.99,14.24
BaggingClassifier,0.99,0.98,0.98,0.99,18.96
ExtraTreeClassifier,0.98,0.97,0.97,0.98,0.18


In [106]:
from sklearn.neighbors import  KNeighborsClassifier
t1=KNeighborsClassifier()
o=t1.fit(X_train,y_train)

In [107]:
y_train

array([18, 12, 16, ..., 18, 16, 18])

In [108]:
o.predict

<bound method KNeighborsClassifier.predict of KNeighborsClassifier()>

In [115]:

my_cl=[]

for i in range(9):

    my_cl.append(  KNeighborsClassifier())
    X=df.iloc[:,1:-9]
    y=df.loc[:,col_names[i]]
    my_cl[i].fit(X,y)



In [116]:
o.predict(X_train)

array([18, 12, 16, ..., 18, 16, 18])

In [117]:
X[0:0]

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity


In [118]:
my_cl

[KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier(),
 KNeighborsClassifier()]

In [119]:
my_cl[0].predict(X)

array([0., 0., 0., ..., 0., 0., 1.])

In [29]:
my_cl[1].predict(X)

array([0., 0., 0., ..., 0., 0., 0.])

In [30]:
my_cl[2].predict(X)

array([1., 1., 1., ..., 1., 1., 0.])

In [120]:
# submission generation
!rm "./Test.csv"

In [122]:
!wget https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv

--2021-12-07 22:18:47--  https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv [following]
--2021-12-07 22:18:48--  https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/Test.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10299206 (9.8M) [text/plain]
Saving to: ‘Test.csv’

Test.csv            100%[===================>]   9.82M  --.-KB/s    in 0.09s   

2021-12-07 22:18:48 (114 MB/s) - ‘Test.csv’ sa

In [123]:
test_df=pd.read_csv("./Test.csv")
test_df.head()

test_df['hdl_cholesterol_human']=0
test_df["hemoglobin(hgb)_human"]=0
test_df["cholesterol_ldl_human"]=0

In [124]:
X_test=test_df.iloc[:,1:-3]

In [125]:
X_test[0:0]

,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,absorbance39,...,absorbance132,absorbance133,absorbance134,absorbance135,absorbance136,absorbance137,absorbance138,absorbance139,absorbance140,absorbance141,absorbance142,absorbance143,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity


In [126]:
from sklearn.neighbors import  KNeighborsClassifier


for i in range(9):

    y_pred=my_cl[i].predict(X_test)
    test_df[col_names[i]]=y_pred
    



In [128]:
y_pred2=o.predict(X_test)

In [127]:
test_df

,Reading_ID,absorbance0,absorbance1,absorbance2,absorbance3,absorbance4,absorbance5,absorbance6,absorbance7,absorbance8,absorbance9,absorbance10,absorbance11,absorbance12,absorbance13,absorbance14,absorbance15,absorbance16,absorbance17,absorbance18,absorbance19,absorbance20,absorbance21,absorbance22,absorbance23,absorbance24,absorbance25,absorbance26,absorbance27,absorbance28,absorbance29,absorbance30,absorbance31,absorbance32,absorbance33,absorbance34,absorbance35,absorbance36,absorbance37,absorbance38,...,absorbance144,absorbance145,absorbance146,absorbance147,absorbance148,absorbance149,absorbance150,absorbance151,absorbance152,absorbance153,absorbance154,absorbance155,absorbance156,absorbance157,absorbance158,absorbance159,absorbance160,absorbance161,absorbance162,absorbance163,absorbance164,absorbance165,absorbance166,absorbance167,absorbance168,absorbance169,temperature,humidity,hdl_cholesterol_human,hemoglobin(hgb)_human,cholesterol_ldl_human,hdl_cholesterol_human_x0_high,hdl_cholesterol_human_x0_low,hdl_cholesterol_human_x0_ok,hemoglobin(hgb)_human_x0_high,hemoglobin(hgb)_human_x0_low,hemoglobin(hgb)_human_x0_ok,cholesterol_ldl_human_x0_high,cholesterol_ldl_human_x0_low,cholesterol_ldl_human_x0_ok
0,ID_37BEI22R,0.45,0.45,0.45,0.46,0.47,0.49,0.49,0.50,0.50,0.51,0.53,0.54,0.55,0.56,0.56,0.56,0.56,0.56,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.54,0.55,0.55,0.55,...,1.32,1.32,1.31,1.31,1.31,1.30,1.29,1.29,1.29,1.29,1.29,1.27,1.27,1.27,1.26,1.25,1.25,1.25,1.22,1.20,1.21,1.16,1.15,1.17,1.16,1.20,40.88,25.70,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00
1,ID_4W85V5DV,0.50,0.51,0.51,0.52,0.53,0.54,0.55,0.56,0.56,0.58,0.59,0.60,0.61,0.61,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.58,0.58,0.59,0.59,0.59,0.60,0.60,...,1.43,1.41,1.40,1.40,1.39,1.38,1.37,1.37,1.36,1.36,1.35,1.35,1.33,1.33,1.34,1.32,1.32,1.32,1.31,1.29,1.27,1.24,1.24,1.22,1.25,1.26,42.35,30.16,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,0.00,0.00,1.00
2,ID_L4YR3NDY,0.44,0.44,0.44,0.45,0.46,0.47,0.47,0.48,0.48,0.50,0.51,0.52,0.53,0.53,0.53,0.53,0.53,0.53,0.53,0.52,0.52,0.52,0.51,0.51,0.51,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.51,...,1.25,1.24,1.23,1.22,1.21,1.21,1.20,1.20,1.20,1.19,1.18,1.17,1.17,1.17,1.17,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.16,1.18,1.17,42.83,25.51,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00
3,ID_U88E3SQ6,0.50,0.51,0.51,0.52,0.53,0.54,0.54,0.54,0.55,0.56,0.57,0.58,0.59,0.59,0.60,0.60,0.60,0.59,0.58,0.58,0.58,0.57,0.57,0.57,0.56,0.56,0.56,0.56,0.55,0.55,0.55,0.55,0.55,0.56,0.56,0.56,0.56,0.57,0.57,...,1.29,1.28,1.27,1.26,1.26,1.25,1.24,1.23,1.23,1.21,1.22,1.21,1.21,1.20,1.20,1.20,1.20,1.18,1.17,1.14,1.11,1.05,1.05,1.02,1.03,1.02,40.95,41.32,0,0,0,0.00,1.00,0.00,0.00,0.00,1.00,1.00,0.00,0.00
4,ID_NW7Z3XU7,0.53,0.53,0.54,0.54,0.55,0.56,0.56,0.57,0.58,0.59,0.60,0.62,0.62,0.63,0.63,0.63,0.63,0.62,0.62,0.62,0.62,0.61,0.61,0.61,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.59,0.59,0.59,0.60,0.60,0.60,...,1.40,1.39,1.39,1.38,1.36,1.35,1.35,1.34,1.33,1.33,1.33,1.31,1.32,1.30,1.30,1.31,1.31,1.31,1.29,1.29,1.30,1.29,1.27,1.28,1.25,1.29,46.99,24.21,0,0,0,0.00,0.00,1.00,0.00,0.00,1.00,1.00,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3655,ID_ADCBL266,0.51,0.51,0.51,0.51,0.52,0.52,0.53,0.54,0.55,0.56,0.57,0.59,0.60,0.60,0.60,0.60,0.60,0.60,0.60,0.59,0.59,0.59,0.59,0.58,0.58,0.58,0.58,0.58,0.57,0.57,0.57,0.57,0.57,0.57,0.58,0.58,0.58,0.59,0.59,...,1.47,1.45,1.45,1.43,1.41,1.42,1.40,1.39,1.39,1.38,1.37,1.36,1.36,1.36,1.34,1.35,1.37,1.35,1.37,1.35,1.37,1.36,1.38,1.37,1.35,1.31,35.54,37.52,0,0,0,0.00,1.00,0.00,0.00,0.00,1.00,0.00,0.00,1.00
3656,ID_

In [38]:
hdl=test_df.loc[:,"hdl_cholesterol_human_x0_high":	"hdl_cholesterol_human_x0_ok"]
hemo=test_df.loc[:,"hemoglobin(hgb)_human_x0_high":	"hemoglobin(hgb)_human_x0_ok"]
col=test_df.loc[:,"cholesterol_ldl_human_x0_high":"cholesterol_ldl_human_x0_ok"]


In [39]:
hdl_deconv=hdl_onehot.inverse_transform(hdl)
hemo_deconv=hemo_onehot.inverse_transform(hemo)
col_deconv=col_onehot.inverse_transform(col)


In [137]:
y_pred_deconv=_3_Label_encoder.inverse_transform(y_pred2)

In [138]:
y_pred_deconv_list=[i.split("+") for i in y_pred_deconv]

In [140]:
y_pred_deconv_list

[['ok', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['ok', 'ok', 'high'],
 ['low', 'ok', 'high'],
 ['ok', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['ok', 'ok', 'ok'],
 ['ok', 'ok', 'high'],
 ['high', 'ok', 'high'],
 ['ok', 'high', 'high'],
 ['low', 'ok', 'high'],
 ['ok', 'ok', 'high'],
 ['ok', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'high', 'ok'],
 ['ok', 'ok', 'ok'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'ok', 'high'],
 ['high', 'ok', 'high'],
 ['high', 'ok', 'high'],
 ['high', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'ok'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'low', 'ok'],
 ['ok', 'high', 'ok'],
 ['ok', 'ok', 'high'],
 ['ok', 'ok', 'ok'],
 ['low', 'ok', 'ok'],
 ['low', 'ok', 'ok'],
 ['low', 'ok', 'ok'],
 ['low', 'ok', 'high'],
 ['ok', 'high', 'ok'],
 ['ok', 'ok', 'high'],
 ['low', 'ok', 'ok'],
 ['low', 'ok', 'ok'],
 

In [144]:
y_pred_deconv_list[:][0]

['ok', 'ok', 'high']

In [146]:
o_df_2=pd.DataFrame(y_pred_deconv_list)

In [147]:
o_df_2.

,0,1,2
0,ok,ok,high
1,ok,ok,ok
2,ok,ok,high
3,low,ok,high
4,ok,ok,high


In [148]:
test_df["hdl_cholesterol_human"]=o_df_2[0]
test_df["hemoglobin(hgb)_human"]=o_df_2[1]
test_df["cholesterol_ldl_human"]=o_df_2[2]



In [149]:
!rm SampleSubmission.csv
!wget https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv

--2021-12-07 22:33:24--  https://github.com/sipocz/Blood/raw/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv [following]
--2021-12-07 22:33:24--  https://raw.githubusercontent.com/sipocz/Blood/9326effc5771a0f4cbeef5d97e329c986c9ba9dd/orig/SampleSubmission.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 384318 (375K) [text/plain]
Saving to: ‘SampleSubmission.csv’

SampleSubmission.cs 100%[===================>] 375.31K  --.-KB/s    in 0.02s   



In [150]:
submission_df=pd.read_csv("./SampleSubmission.csv")

In [151]:
submission_df.head(1)

,Reading_ID,target
0,ID_00902R9H_hdl_cholesterol_human,nan


In [152]:
outdf=test_df
outdf.set_index("Reading_ID",inplace=True)

In [153]:
outdf.loc["ID_UPO13B7Z"]

absorbance0                     0.49
absorbance1                     0.50
absorbance2                     0.50
absorbance3                     0.51
absorbance4                     0.52
                                ... 
hemoglobin(hgb)_human_x0_low    0.00
hemoglobin(hgb)_human_x0_ok     1.00
cholesterol_ldl_human_x0_high   0.00
cholesterol_ldl_human_x0_low    0.00
cholesterol_ldl_human_x0_ok     1.00
Name: ID_UPO13B7Z, Length: 184, dtype: object

In [154]:
submission_df.head(1)

,Reading_ID,target
0,ID_00902R9H_hdl_cholesterol_human,nan


In [155]:
for i in submission_df.index:
    #print(i)
    indexer=submission_df.loc[i,"Reading_ID"]

    splitter=indexer.split("_",2)
    out_index=splitter[0]+"_"+splitter[1]
    col_index=splitter[2]
    print(out_index,col_index)
    result=outdf.loc[out_index][col_index]
    submission_df.iloc[i,1]=result

A streamkimeneten csak az utolsó 5000 sor látható.
ID_JVY48D8U cholesterol_ldl_human
ID_JVY48D8U hemoglobin(hgb)_human
ID_JW4F66XM cholesterol_ldl_human
ID_JW4F66XM hdl_cholesterol_human
ID_JW4F66XM hemoglobin(hgb)_human
ID_JWXML3KU cholesterol_ldl_human
ID_JWXML3KU hemoglobin(hgb)_human
ID_JWXML3KU hdl_cholesterol_human
ID_JXCWA9OK cholesterol_ldl_human
ID_JXCWA9OK hemoglobin(hgb)_human
ID_JXCWA9OK hdl_cholesterol_human
ID_JY2WRCO4 hemoglobin(hgb)_human
ID_JY2WRCO4 hdl_cholesterol_human
ID_JY2WRCO4 cholesterol_ldl_human
ID_JYFHLCK2 cholesterol_ldl_human
ID_JYFHLCK2 hemoglobin(hgb)_human
ID_JYFHLCK2 hdl_cholesterol_human
ID_JYUWNU78 cholesterol_ldl_human
ID_JYUWNU78 hemoglobin(hgb)_human
ID_JYUWNU78 hdl_cholesterol_human
ID_JYV38755 hemoglobin(hgb)_human
ID_JYV38755 cholesterol_ldl_human
ID_JYV38755 hdl_cholesterol_human
ID_JYYFUL4N cholesterol_ldl_human
ID_JYYFUL4N hdl_cholesterol_human
ID_JYYFUL4N hemoglobin(hgb)_human
ID_JZ2BEIAY cholesterol_ldl_human
ID_JZ2BEIAY hdl_cholesterol_hum

In [156]:
submission_df.set_index("Reading_ID", inplace=True)

In [157]:
submission_df

,target
Reading_ID,
ID_00902R9H_hdl_cholesterol_human,low
ID_00902R9H_hemoglobin(hgb)_human,ok
ID_00902R9H_cholesterol_ldl_human,ok
ID_00FQS7AO_hdl_cholesterol_human,ok
ID_00FQS7AO_hemoglobin(hgb)_human,ok
...,...
ID_ZZKK303Z_hdl_cholesterol_human,ok
ID_ZZKK303Z_cholesterol_ldl_human,ok
ID_ZZRYJBQJ_hdl_cholesterol_human,high


In [158]:

    

def create_submission_name(project):
    from datetime  import datetime
    strdate=datetime.now().strftime("%Y%m%d_%H%M%S")
    out=project+"_"+"submission"+"_"+strdate+".csv"
    return(out)




In [159]:
submission_df.head()

,target
Reading_ID,
ID_00902R9H_hdl_cholesterol_human,low
ID_00902R9H_hemoglobin(hgb)_human,ok
ID_00902R9H_cholesterol_ldl_human,ok
ID_00FQS7AO_hdl_cholesterol_human,ok
ID_00FQS7AO_hemoglobin(hgb)_human,ok


In [160]:
submission_fname=create_submission_name("BloodAI")
print(submission_fname)


BloodAI_submission_20211207_223425.csv


In [162]:
submission_df.to_csv(submission_fname,index=True)


In [163]:
!head $submission_fname

Reading_ID,target
ID_00902R9H_hdl_cholesterol_human,low
ID_00902R9H_hemoglobin(hgb)_human,ok
ID_00902R9H_cholesterol_ldl_human,ok
ID_00FQS7AO_hdl_cholesterol_human,ok
ID_00FQS7AO_hemoglobin(hgb)_human,ok
ID_00FQS7AO_cholesterol_ldl_human,ok
ID_00QJTIBO_hdl_cholesterol_human,low
ID_00QJTIBO_hemoglobin(hgb)_human,ok
ID_00QJTIBO_cholesterol_ldl_human,ok
